In [1]:
%matplotlib inline

# PFN2022 夏季国内インターン テーマ別課題【工学応用】

与えられたデータセットを用いて、２次元空間上の単調な関数を予測する機械学習モデルを作成してください。以下の指示に従ってこの notebook に説明や実装などを追加し、編集した notebook (.ipynb ファイル) をレポートとして提出してください。

Notebook には Python コードとその出力だけではなく、Markdown テキストを含めることができます。解答にあたっては、思考の過程がわかるように、適切にテキストやコメントを挿入してください。その際、解答にあたって参考にした文献は適切に引用してください。
解答欄のセルは必要に応じて挿入・追加して構いません。

## 問題

あるグルメレビューサイトでは、様々なレストランを検索したり、そのレストランに関する評価やレビューコメントをつけたりすることができます。

ユーザがレストランを検索したとき、検索結果として表示されたそれぞれのレストランのリンクをクリックする確率を推定してください。

### データセット

本課題では、上記の状況を想定して、人工的に生成したデータセットを用います。つぎの３つの CSV ファイルを提供します。

- `train.csv` - 訓練データ。
- `valid.csv` - 検証データ。
- `test.csv` - テストデータ。

訓練データ・検証データは、ユーザの過去の行動に基づき、検索結果として表示されやすい (ユーザーに好まれやすい) レストランが多くサンプルされています。訓練・検証データは、事前に入手されたものとして、自由に分析したり、機械学習モデルの訓練に用いたりして構いません。

本課題では、機械学習モデルを実際に適用した場面を想定した評価まで行うために、テストデータも合わせて提供します。
ユーザーに必ずしも好まれないレストランを含む、様々なレストランで評価するために、テストデータはユーザの好みに依らない方法でサンプルされています。

それぞれの CSV ファイルは、アクセスに対して表示されたレストランに関する情報が格納されています。
各行は、あるアクセスで表示されたあるレストランについての情報を表します。
検索アクセス単位のデータであるため、同一のレストランが複数行に渡って出現することがあります。
また、CSV ファイルにはつぎの 3 つのカラムがあります。

- `avg_rating` - レストランの平均評価 (Average Rating)
- `num_reviews` - レストランについているレビューの件数 (Number of Reviews)
- `clicked` - あるアクセスに対してそのレストランがクリックされたかどうか

それぞれの行に対して、`avg_rating` と `num_reviews` から `clicked` を予測します。

機械学習を応用する一般的な状況では、テストデータが与えられなかったり、テストデータに関する予測対象の値 (正解) が与えられないことが通常です。
テストデータのうち、`avg_rating` と `num_reviews` (予測の入力) は、事前に入手されたものとして自由に分析して構いませんが、`clicked` カラム (予測対象) を直接観察・分析したり、機械学習の訓練に利用してはいけません。

### 予測の評価

本問題で扱うタスクは `clicked` が 0 と 1 のどちらであるかを推定する二値分類問題です。
予測性能は、二値分類の評価に一般的に用いられる指標の一つである、受信者動作特性曲線 (receiver operating characteristic curve, ROC) の曲線下面積 (area under the curve, AUC) を用いて評価してください。

ROC AUC は、たとえば scikit-learn の [sklearn.metrics.roc_auc_score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html) を用いて計算できます。

### notebook の実行環境

本課題で扱うデータとモデルは十分軽量であり、GPU などの高性能な計算デバイスを用いなくても十分遂行可能と思われます。

提出された notebook は Python 3.9 環境で動作確認と評価を行います。

この notebook ではデフォルトで numpy、pandas、pytorch、scikit-learn を import しています。
解答上の必要に応じてその他のライブラリをさらに import することは差し支えありません。
その際、どのようなライブラリを選択・利用したかについても評価の対象となります。

In [2]:
import numpy as np
import pandas as pd
import sklearn
import torch

## 問1

二値分類の評価に用いられる [ROC](https://scikit-learn.org/stable/modules/model_evaluation.html#roc-metrics) とはどのような曲線か、次の点を説明してください。

- 横軸と縦軸の意味
- プロットの方法
- AUC の値の解釈方法

【問1の解答はここに Markdown テキストで記入してください】

## 問2

それではデータセットのファイルを開き、データの観察から始めましょう。まずは Pandas を用いて３つの CSV ファイルを開きます。

In [3]:
data_train = pd.read_csv("train.csv")
data_valid = pd.read_csv("valid.csv")
data_test = pd.read_csv("test.csv")

たとえば訓練データは次のようなテーブル形式になっていることがわかります。

In [4]:
data_train

,avg_rating,num_reviews,clicked
0,2.547957,19.0,1
1,3.870380,174.0,1
2,3.870380,174.0,0
3,3.870380,174.0,1
4,3.870380,174.0,0
...,...,...,...
231,1.586990,51.0,1
232,1.586990,51.0,1
233,2.980449,138.0,0
234,3.540500,129.0,1


以下の分析では `data_test` (テストデータ) の `clicked` カラム (予測対象) を用いてはいけません。

### 問2-1

`data_train`、`data_valid`、`data_test` の3つのデータセットについて、`avg_rating` と `num_reviews` の分布を可視化してください。

In [ ]:
# 【このセルを用いて可視化してください】

### 問2-2

データの分布を可視化した図を観察し、データの統計的性質について議論してください。
また、機械学習モデルの訓練・検証・テストにおいて予想される課題あるいは注意点を説明してください。

【問2-2の解答はここに Markdown テキストで記入してください】

## 問3

`clicked` を予測する機械学習モデルを一つ作成してみましょう。この問で作成するモデルを以下 Baseline モデルとよびます。

つぎの仕様に従う Baseline モデルを実装・訓練・評価しましょう。

- 使用する機械学習モデルは多層パーセプトロン (multi layer perceptron, MLP) とする。
- Baseline は次の構造をもつ 4 層 MLP とする。いずれの層も全結合層である。
    - 入力層: 2 次元 (`avg_rating` および `num_reviews` に相当)
    - 隠れ層#1: 16 次元
    - 隠れ層#2: 8 次元
    - 隠れ層#3: 8 次元
    - 出力層: 1 次元 (`clicked` に相当)
- Baseline の隠れ層で用いる活性化関数はすべて正規化線形関数 (rectified linear unit, ReLU) とする。
- Baseline の実装には [Pytorch](https://pytorch.org/docs/stable/index.html) を用いること。

### 問3-1

上記仕様に従う Baseline を Python のクラスとして実装してください。

以下に Pytorch で実装するテンプレートを示します。足りない部分を補って実装を完成させてください。

In [ ]:
from torch import nn, Tensor

class Baseline(nn.Module):
    def __init__(self) -> None:
        super().__init__()

        # TODO: ここに実装を追加してください

    def forward(self, x: Tensor) -> None:
        # x: Tensor of shape (B, 2)
        batch_size = x.shape[0]
        assert x.shape == (batch_size, 2)

        # TODO: ここに実装を追加してください

        # output: Tensor of shape (B, 1)
        assert output.shape == (batch_size, 1)
        return output

### 問3-2

Baseline を二値分類器として訓練するための損失関数を実装してください。

以下に Pytorch で実装するテンプレートを示します。足りない部分を補って実装を完成させてください。

In [ ]:
def loss_fn(model: Baseline, x: Tensor, y: Tensor) -> Tensor:
    # x: Tensor of shape (B, 2)
    # y: Tensor of shape (B, )
    batch_size = x.shape[0]
    assert x.shape == (batch_size, 2)
    assert y.shape == (batch_size,)

    # TODO: ここに実装を追加してください

    # loss: Tensor of shape (1,) or 0-dim tensor
    return loss

### 問3-3

Baseline を用いて、バッチ入力に対する `clicked` の確率を予測する関数を実装してください。

以下に Pytorch で実装するテンプレートを示します。足りない部分を補って実装を完成させてください。

In [ ]:
def predict(model: Baseline, x: Tensor) -> Tensor:
    # x: Tensor of shape (B, 2)
    batch_size = x.shape[0]
    assert x.shape == (batch_size, 2)

    # TODO: ここに実装を追加してください

    # output: Tensor of shape (B, 1)
    assert prediction.shape == (batch_size, 1)
    return prediction

### 問3-4

Baseline を訓練・検証するための関数を実装してください。

以下に Pytorch で実装するテンプレートを示します。足りない部分を補って実装を完成させてください。

In [ ]:
from typing import Literal

from torch.optim import Optimizer
from torch.utils.data import DataLoader

def train(
    model: Baseline,
    device: Literal["cpu", "cuda"],
    train_loader: DataLoader,
    optimizer: Optimizer,
) -> float:
    model.train()
    loss_sum = 0
    for x, y in train_loader:
        x, y = x.to(device), y.to(device)

        # TODO: ここに実装を追加してください

        loss_sum += loss.item()
    return loss_sum / len(train_loader.dataset)


def valid(
    model: Baseline,
    device: Literal["cpu", "cuda"],
    eval_loader: DataLoader,
) -> float:
    model.eval()
    loss_sum = 0
    with torch.no_grad():
        for x, y in eval_loader:
            x, y = x.to(device), y.to(device)
            loss = loss_fn(model, x, y)
            loss_sum += loss.item()
    return loss_sum / len(eval_loader.dataset)

### 問3-5

ここまでに実装した Baseline を訓練し、検証データ・テストデータに対する予測性能の評価値として ROC AUC を求めてください。

以下に Pytorch の実装を示します。この実装は改変せず、そのまま使用してください。

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score


# training settings
device = "cpu"
model = Baseline().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# dataset
scaler = StandardScaler()
X_train = scaler.fit_transform(data_train[["avg_rating", "num_reviews"]].to_numpy())
X_valid = scaler.transform(data_valid[["avg_rating", "num_reviews"]].to_numpy())
X_test = scaler.transform(data_test[["avg_rating", "num_reviews"]].to_numpy())

train_loader = torch.utils.data.DataLoader(
    torch.utils.data.TensorDataset(
        torch.tensor(X_train, dtype=torch.float32),
        torch.tensor(data_train["clicked"], dtype=torch.float32),
    ),
    batch_size=64,
    shuffle=True,
    drop_last=True,
)
valid_loader = torch.utils.data.DataLoader(
    torch.utils.data.TensorDataset(
        torch.tensor(X_valid, dtype=torch.float32),
        torch.tensor(data_valid["clicked"], dtype=torch.float32),
    ),
    batch_size=64,
)

# run training loop
epoch_size = 200
log_epoch_interval = 10
print("Epoch\ttrain loss\tvalid loss")
for epoch in range(1, epoch_size + 1):
    train_loss = train(model, device, train_loader, optimizer)
    valid_loss = valid(model, device, valid_loader)
    if epoch % log_epoch_interval == 0:
        print(f"{epoch}\t{train_loss:.6f}\t{valid_loss:.6f}")


def predict_to_dataset(
    model: Baseline,
    X: np.ndarray,
) -> np.ndarray:
    loader = torch.utils.data.DataLoader(
        torch.utils.data.TensorDataset(
            torch.tensor(X, dtype=torch.float32),
        ),
        batch_size=64,
    )
    model.eval()
    with torch.no_grad():
        pred = []
        for x, in loader:
            x = x.to(device)
            pred.append(predict(model, x).detach().cpu().numpy())
        return np.concatenate(pred, axis=0)


print("Validation AUC:", roc_auc_score(data_valid["clicked"], predict_to_dataset(model, X_valid)))
print("Test AUC:", roc_auc_score(data_test["clicked"], predict_to_dataset(model, X_test)))

### 問3-6

ここまでに実装した Baseline モデルは、モデル内部変数の初期化がランダムに実行されるため、機械学習の訓練には必ずしも再現性がありません。
すなわち、Baseline モデルを全く同じプログラムを用いて訓練したとしても、常に同じ予測性能の機械学習モデルが得られるとは限りません。

以下に、Baseline モデルを独立に 10 回作成・訓練・評価するためのプログラムを示します。
このプログラムを改変せずそのまま用いて、10 回分の訓練結果を出力してください。

In [ ]:
models = []
for model_index in range(10):
    print(f"# Model {model_index}")
    model = Baseline().to(device)
    models.append(model)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

    epoch_size = 200
    for epoch in range(1, epoch_size + 1):
        train_loss = train(model, device, train_loader, optimizer)
        valid_loss = valid(model, device, valid_loader)
    print(f"Train Loss {train_loss:.6f}\tValid Loss {valid_loss:.6f}\tAt Epoch {epoch_size}")

    valid_auc = roc_auc_score(data_valid["clicked"], predict_to_dataset(model, X_valid))
    test_auc = roc_auc_score(data_test["clicked"], predict_to_dataset(model, X_test))
    print(f"Validation AUC: {valid_auc}\tTest AUC: {test_auc}")
    print()

### 問3-7

問3-6 の出力を観察し、Baseline モデルの訓練のロバスト性および予測性能について説明してください。
さらに、ここまでの問を通して作成してきた Baseline モデルがもつ問題点と改善策について議論してください。

【問3-7の解答はここに Markdown テキストで記入してください】

## 問4

あなたがこのデータを分析していると、それを後ろから見ていた先輩が、こんなことを言ってきました。

> 検索して出てきたレストランって、評価が高かったらいい店なんだろうし、レビューが多かったら人気のある店なんだろうから、期待してクリックしたくなるよね？

そこであなたは、検索結果として表示されたレストランがクリックされる確率は、平均評価 `avg_rating` の高さや、レビューコメントの数 `num_reviews` と相関があると考え、このヒューリスティックを用いて Baseline モデルを改善してみることにしました。
予測として出力される `clicked` の確率が、`avg_rating` や `num_reviews` に関して単調増加であることを保証するようなモデルを作成してみましょう。

### 問4-1

予測モデルの単調増加性を保証するアプローチを調査し、レポートしてください。

【問4-1の解答はここに Markdown テキストで記入してください】

### 問4-2

予測として出力される `clicked` の確率が、`avg_rating` や `num_reviews` に関して単調増加であることを保証するようなモデルを実装・訓練・評価し、その結果について議論してください。

問3 までに作成した実装をそのまま、または改変して、再利用することは差し支えありません。
単調増加性の保証に関する複数の方針を実装・評価しても構いません。

レポート評価にあたっては、最終的に得られた予測性能の良さや、試した方針の数よりもむしろ、調査・実装の過程や内容およびその説明を重視します。
解答にあたっては、思考の過程がわかるように、適切にテキストやコメントを挿入してください。
その際、解答にあたって参考にした文献は適切に引用してください。

以下、自由にセルを追加して構いません。